In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
base_path = '../../'
sys.path.append(base_path)
from src.dgl_flow_field_dataset import DGLSurfaceFlowFieldDataset
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset

In [16]:
ds_pv = PyvistaFlowFieldDataset(os.path.join(base_path,'datasets/Example cgns volume + surface'))
ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'),ds_pv)
#ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'))

In [10]:
ds_pv[0].get_surface_points(block_index=1)

pyvista_ndarray([[ 0.05844221, -0.03099999,  0.00883333],
                 [ 0.05844221, -0.03099999,  0.01      ],
                 [ 0.05649415, -0.03099999,  0.01      ],
                 ...,
                 [ 0.05002494, -0.02901889,  0.00283333],
                 [ 0.05002494, -0.02712015,  0.00083333],
                 [ 0.05002494, -0.02522142,  0.        ]], shape=(868, 3))

In [15]:
ds_pv[1].plot_volume("Pressure")

Widget(value='<iframe src="http://localhost:43353/index.html?ui=P_0x7fd841db6f50_4&reconnect=auto" class="pyvi…

In [17]:
ds_dgl[0]

Graph(num_nodes=21293, num_edges=83778,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [19]:
from dgl.dataloading import GraphDataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloader = GraphDataLoader(ds_dgl, batch_size=1, shuffle=True)
for batch in dataloader:
    print(batch)
    

Graph(num_nodes=22337, num_edges=87616,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_nodes=21293, num_edges=83778,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_node

In [20]:
import dgl


def get_node_edge_X(graph: dgl.DGLGraph):
    node_X = torch.cat([graph.ndata["Position"], graph.ndata["Normal"]],dim=1)
    edge_X = torch.cat([graph.edata["dx"]],dim=1)
    return node_X, edge_X

def get_node_Y(graph: dgl.DGLGraph):
    return torch.cat([graph.ndata["Pressure"].unsqueeze(1),graph.ndata["Temperature"].unsqueeze(1),graph.ndata['ShearStress']],dim=1)

def set_graph_features(graph: dgl.DGLGraph, node_X, edge_X, node_Y):
    graph.ndata["Position"] = node_X[:,:3]
    graph.ndata["Normal"] = node_X[:,3:]
    graph.edata["dx"] = edge_X
    graph.ndata["Pressure"] = node_Y[:,0]
    graph.ndata["Temperature"] = node_Y[:,1]
    graph.ndata["ShearStress"] = node_Y[:,2:]
g=ds_dgl[0]
g_cp=g.clone()
ndx, edx = get_node_edge_X(g)
ndy = get_node_Y(g)
set_graph_features(g_cp, ndx, edx, ndy)
assert torch.allclose(g_cp.ndata["Position"], g.ndata["Position"])
assert torch.allclose(g_cp.ndata["Normal"], g.ndata["Normal"])
assert torch.allclose(g_cp.edata["dx"], g.edata["dx"])
assert torch.allclose(g_cp.ndata["Pressure"], g.ndata["Pressure"])
assert torch.allclose(g_cp.ndata["Temperature"], g.ndata["Temperature"])
assert torch.allclose(g_cp.ndata["ShearStress"], g.ndata["ShearStress"])
num_node_features = ndx.shape[1]
num_edge_features = edx.shape[1]
num_node_labels = ndy.shape[1]
print("Node X: ",ndx, ndx.shape)
print("Edge X: ",edx, edx.shape)
print("Node Y: ",ndy, ndy.shape)

Node X:  tensor([[ 1.2582e+00, -1.7498e+00, -1.0662e+00, -2.0974e-03, -3.8745e-03,
         -1.0238e+00],
        [ 1.2585e+00, -1.6781e+00, -1.0662e+00, -2.0974e-03, -3.8745e-03,
         -1.0238e+00],
        [ 1.2408e+00, -1.7498e+00, -1.0662e+00, -2.0974e-03, -3.8745e-03,
         -1.0238e+00],
        ...,
        [ 5.1143e-01, -3.1371e-02, -2.3643e-01, -2.0974e-03,  4.0907e+00,
          1.5715e-02],
        [ 4.9444e-01, -1.3117e-02, -6.9371e-03, -2.0974e-03, -3.8745e-03,
         -1.0238e+00],
        [ 4.9444e-01, -3.1371e-02, -6.8725e-02, -2.0974e-03,  4.0907e+00,
          1.5715e-02]]) torch.Size([21293, 6])
Edge X:  tensor([[ 1.9371e-02,  1.4985e+00, -5.3246e-10],
        [-1.4207e+00,  6.2623e-11, -5.3246e-10],
        [ 1.4207e+00,  6.2623e-11, -5.3246e-10],
        ...,
        [ 1.3838e+00,  6.2623e-11, -5.3246e-10],
        [ 6.4081e-10,  6.2623e-11, -4.8453e+00],
        [-1.3838e+00,  6.2623e-11, -5.3246e-10]]) torch.Size([83778, 3])
Node Y:  tensor([[-0.5311,  0.42

In [21]:
from modulus.models.meshgraphnet import MeshGraphNet
model = MeshGraphNet(
    input_dim_nodes=num_node_features,
    input_dim_edges=num_edge_features,
    output_dim=num_node_labels,
    aggregation='sum',
    hidden_dim_edge_encoder=64,
    hidden_dim_node_encoder=64,
    hidden_dim_processor=64,
    hidden_dim_node_decoder=64
)
model=model.to(device)
#model(ndx.to(device),edx.to(device),ds_dgl[0].to(device))

In [22]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.amp import GradScaler
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99985 ** epoch)
scaler = GradScaler(device=device)

In [23]:
def format_vector(v):
    return ' '.join([f'{x:.2f}' for x in v])

In [24]:

from modulus.launch.utils import save_checkpoint, load_checkpoint
checkpoint_path = 'checkpoints'
os.makedirs(checkpoint_path,exist_ok=True)
epoch_init = load_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,device=device)
num_epochs = 200
model.to(device)
for epoch in range(epoch_init,num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        node_X, edge_X = get_node_edge_X(batch)
        node_Y = get_node_Y(batch)
        node_Y_pred = model(node_X,edge_X,batch)
        batch_pred_graph = batch.clone()
        set_graph_features(batch_pred_graph, node_X, edge_X, node_Y_pred)

        agg_force_pred = ds_dgl.compute_aggregate_force(batch_pred_graph)
        agg_force = ds_dgl.compute_aggregate_force(batch)
        #print('Agg force pred: ',format_vector(agg_force_pred.tolist()),' Agg force: ',format_vector(agg_force.tolist()))
        loss = torch.nn.functional.mse_loss(node_Y_pred,node_Y) + 5* torch.nn.functional.mse_loss(agg_force_pred,agg_force)
        total_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print(f'Epoch {epoch} loss: {total_loss}')
    scheduler.step()
    if epoch % 10 == 0:
        save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,epoch)
save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,num_epochs)

`DistributedManager` not initialized already. Initializing now, but this might lead to unexpected errors
/home/olep/Documents/software/master/flow_field_dataset/.venv/lib/python3.11/site-packages/modulus/distributed/manager.py:346: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(


Epoch 199 loss: 1.036153495311737


In [27]:
model.to(device)
g=ds_dgl[0].to(device)
g_pred = g.clone().to(device)
ndx, edx = get_node_edge_X(g)
with torch.no_grad():
    y_pred = model(ndx,edx,g)
set_graph_features(g_pred, ndx, edx, y_pred)
ds_dgl.plot_surface(g_pred,"Temperature")

Widget(value='<iframe src="http://localhost:43353/index.html?ui=P_0x7fd7d8cc7fd0_7&reconnect=auto" class="pyvi…

In [32]:
ds_dgl.plot_surface(g,"Temperature", object_id=1)

Widget(value='<iframe src="http://localhost:43353/index.html?ui=P_0x7fd7d21f2c50_9&reconnect=auto" class="pyvi…

In [31]:
ds_dgl.compute_aggregate_force(g, 1)

tensor([1.0340e-02, 5.6809e-05, 6.3741e-03], device='cuda:0')

In [33]:
ds_dgl.compute_aggregate_force(g_pred,1)

tensor([ 0.0115, -0.0007,  0.0030], device='cuda:0')